In [1]:
import pandas as pd
import matplotlib.pyplot as plt
#pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Preparing the dataset


file_name_1 = 'C:/temp/export-address-token-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_1.csv'
file_name_2 = 'C:/temp/export-address-token-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_2.csv'
df_1 = pd.read_csv(file_name_1)  
df_2 = pd.read_csv(file_name_2) 
df_steth= pd.concat([df_1, df_2],ignore_index=True)

df_steth['Value'] = pd.to_numeric(df_steth['Value'].str.replace(',', ''))
df_steth.info()


file_name_3 = 'C:/temp/export-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_1.csv'
file_name_4 = 'C:/temp/export-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_2.csv'
df_3 = pd.read_csv(file_name_3)  
df_4 = pd.read_csv(file_name_4) 
df_internal= pd.concat([df_3, df_4],ignore_index=True)
df_internal = df_internal[df_internal['ErrCode']!='Out of gas']
df_internal.info()

file_name_5 = 'C:/temp/export-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_withnotes_1.csv'
file_name_6 = 'C:/temp/export-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_withnotes_2.csv'
df_5 = pd.read_csv(file_name_5)  
df_6 = pd.read_csv(file_name_6) 
df_trans= pd.concat([df_5, df_6],ignore_index=True)
df_trans = df_trans[df_trans['ErrCode']!='Out of gas']
df_trans.info()

file_name_7 = 'C:/temp/export-token-0x06325440d014e39736583c165c2963ba99faf14e_1.csv'
file_name_8 = 'C:/temp/export-token-0x06325440d014e39736583c165c2963ba99faf14e_2.csv'
df_7 = pd.read_csv(file_name_7)  
df_8 = pd.read_csv(file_name_8) 
df_stecrv= pd.concat([df_7, df_8],ignore_index=True)
df_stecrv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5568 entries, 0 to 5567
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Txhash           5568 non-null   object 
 1   UnixTimestamp    5568 non-null   int64  
 2   DateTime         5568 non-null   object 
 3   From             5568 non-null   object 
 4   To               5568 non-null   object 
 5   Value            5568 non-null   float64
 6   ContractAddress  5568 non-null   object 
 7   TokenName        5568 non-null   object 
 8   TokenSymbol      5568 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 391.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5935 entries, 0 to 5934
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Txhash                       5935 non-null   object 
 1   Blockno                      5935 non-

In [3]:
# steth sold
adr_sold_steth = df_steth[df_steth['From']!='0xdc24316b9ae028f1497c275eb9192a3ea0f67022'][['Txhash','DateTime','From','Value']] 
adr_sold_steth.columns = ['Txhash','DateTime','Address','Value_steth']
adr_sold_steth['Value_steth'] = adr_sold_steth['Value_steth']*(-1)

# steth bought
adr_bought_steth = df_steth[df_steth['To']!='0xdc24316b9ae028f1497c275eb9192a3ea0f67022'][['Txhash','DateTime','To','Value']] 
adr_bought_steth.columns = ['Txhash','DateTime','Address','Value_steth']

# steth turnover
adr_steth= pd.concat([adr_bought_steth, adr_sold_steth],ignore_index=True).sort_values(by='DateTime') 
adr_list = adr_steth['Txhash'].unique().tolist()

# eth bought (when steth sold)
adr_bought_eth  = df_internal.query('Txhash in @adr_list')[['Txhash', 'DateTime','TxTo','Value_OUT(ETH)']]
adr_bought_eth.columns = ['Txhash','DateTime','Address','Value_eth']

# eth sold (when steth bought)
adr_sold_eth = df_trans.query('Txhash in @adr_list')[['Txhash', 'DateTime','From','Value_IN(ETH)']]
adr_sold_eth.columns = ['Txhash','DateTime','Address','Value_eth']
adr_sold_eth['Value_eth'] = adr_sold_eth['Value_eth']*(-1)

# eth turnover
adr_eth= pd.concat([adr_bought_eth, adr_sold_eth],ignore_index=True).sort_values(by='DateTime') 
adr_eth = adr_eth.pivot_table(index = ['Txhash','DateTime','Address'], values = 'Value_eth', aggfunc='sum').reset_index()

# sterth + eth turnover together
adr_steth_eth = pd.merge(adr_steth,adr_eth, on = ['Txhash','DateTime','Address'])

# check add/remove liqiudity
adr_stecrv_add = df_stecrv.query('Txhash in @adr_list')[df_stecrv.query('Txhash in @adr_list')['From'] == '0x0000000000000000000000000000000000000000'][['Txhash', 'DateTime', 'To','Quantity']]
adr_stecrv_add.columns = ['Txhash','DateTime','Address','Value_stecrv']
adr_stecrv_remove = df_stecrv.query('Txhash in @adr_list')[df_stecrv.query('Txhash in @adr_list')['To'] == '0x0000000000000000000000000000000000000000'][['Txhash', 'DateTime','From','Quantity']]
adr_stecrv_remove['Quantity']=adr_stecrv_remove['Quantity'] *(-1)
adr_stecrv_remove.columns = ['Txhash','DateTime','Address','Value_stecrv']
adr_stecrv= pd.concat([adr_stecrv_add, adr_stecrv_remove],ignore_index=True).sort_values(by='DateTime') 

# the full dataset of transactions: sterth + eth + stecrv turnover together
adr_full = pd.merge(adr_steth_eth,adr_stecrv, on = ['Txhash','DateTime','Address'], how='left')

# steth exchange price calculation
adr_full['steth_price']=adr_full['Value_eth']/adr_full['Value_steth']*(-1)

# adding type of transactions
result1=[]
result2=[]
for row in adr_full[['Value_steth','Value_eth', 'Value_stecrv']].to_numpy():
    if row[2]>0:
        result1.append('add_liquidity')
        result2.append('liquidity')
    elif row[2]<0:
        result1.append('remove_liquidity')
        result2.append('liquidity')
    elif row[0] >0 and row[1] < 0:
        result1.append('steth_bought') 
        result2.append('exchange')
    elif row[0] <0 and row[1] > 0:
        result1.append('steth_sold')
        result2.append('exchange')
    else:
        result1.append('tbd')
        result2.append('tbd')
adr_full['type_1'] =  result1 
adr_full['type_0'] =  result2 

# addresses that both sold and bought steth (= made arbing)
list_sold = adr_full[adr_full['type_1']=='steth_sold']['Address'].unique().tolist()
list_bought = adr_full[adr_full['type_1']=='steth_bought']['Address'].unique().tolist()
list_ex = (set(list_sold)&set(list_bought))

# dataset with addresses that both sold and bought steth
adr_ex=adr_full.query('Address in @list_ex')

# adding the transactions' fees
list_txhash_ex = adr_ex['Txhash'].tolist()
df_trans.query('Txhash in @list_txhash_ex')[['Txhash','TxnFee(ETH)']]
adr_full_ex = pd.merge(adr_ex,df_trans.query('Txhash in @list_txhash_ex')[['Txhash','TxnFee(ETH)']], on = 'Txhash')
adr_full_ex['TxnFee(ETH)'] = adr_full_ex['TxnFee(ETH)'] *(-1)
adr_full_ex.sort_values(by = ['Address','DateTime'])

# the outcome: addresses that made arbing and their exchange transactions and financial results 
dic_resume_all = {} # dataset: all addresses that both sold and bought steth and their financial results
dic_trans = {} # dataset: addresses that both sold and bought stethg and their exchange transactions
list_resume_arbing = [] # list of addresses that made arbing
list_resume_arbing_leaders = [] # list of addresses-leaders that made arbing

for address in list_ex:
    # getting exchange transactons for the address 
    type_adr = 'exchange'
    df_adr_arb =adr_full_ex.query('Address == @address and type_0 == @type_adr')
    # margin as the difference of weighted by stethtotal value price for selling and buying 
    margin = abs(df_adr_arb[df_adr_arb['type_1']=='steth_sold']['Value_eth'].sum()/df_adr_arb[df_adr_arb['type_1']=='steth_sold']['Value_steth'].sum()) - abs(df_adr_arb[df_adr_arb['type_1']=='steth_bought']['Value_eth'].sum()/df_adr_arb[df_adr_arb['type_1']=='steth_bought']['Value_steth'].sum())
    # value_steth_for_calc as min steth total value of selling and buying
    value_steth_for_calc = min(abs(df_adr_arb[df_adr_arb['type_1']=='steth_sold']['Value_steth'].sum()),abs(df_adr_arb[df_adr_arb['type_1']=='steth_bought']['Value_steth'].sum()))
    gross_profit = value_steth_for_calc*margin
    # fees_paid are taken proportionally the min steth total value of selling and buying
    fees_paid = df_adr_arb[df_adr_arb['type_1']=='steth_sold']['TxnFee(ETH)'].sum()*value_steth_for_calc/abs(df_adr_arb[df_adr_arb['type_1']=='steth_sold']['Value_steth'].sum()) + df_adr_arb[df_adr_arb['type_1']=='steth_bought']['TxnFee(ETH)'].sum()*value_steth_for_calc/abs(df_adr_arb[df_adr_arb['type_1']=='steth_bought']['Value_steth'].sum())
    operating_profit = gross_profit+fees_paid
    # time_period in seconds, APR and net_APR in %
    time_period = df_trans[df_trans['Txhash'] == df_adr_arb.iloc[len(df_adr_arb)-1]['Txhash']]['UnixTimestamp'].sum() - df_trans[df_trans['Txhash'] == df_adr_arb.iloc[0]['Txhash']]['UnixTimestamp'].sum()
    APR = gross_profit/value_steth_for_calc / time_period *(365*24*60*60*100)
    net_APR = operating_profit/value_steth_for_calc / time_period *(365*24*60*60*100)
    
    adr_resume = pd.DataFrame({'characteristics':['Value of exchange, stETH',
                                                 'Margin of exchange, ETH',
                                                 'Gross profit, ETH:',
                                                 'APR, %:',
                                                 'Fees paid, ETH:',
                                                 'Operating profit, ETH:',
                                                 'Net APR, %:'],
                              'values':[value_steth_for_calc,
                                       margin,
                                       gross_profit,
                                       APR,
                                       fees_paid,
                                       operating_profit,
                                       net_APR]})#.set_index('characteristics')
    
    # dataset: all addresses that both sold and bought steth and their financial results
    dic_resume_all.update({address:adr_resume})
    
    # dataset: addresses that both sold and bought stethg and their exchange transactions
    dic_trans.update({address:df_adr_arb[['Txhash','DateTime', 'Value_steth', 'Value_eth', 'TxnFee(ETH)', 'steth_price', 'type_1']].set_index('Txhash')})
    
    # list of addresses that made arbing: margin > 0
    if margin > 0: 
        list_resume_arbing.append(address)
        
    # list of addresses-leaders that made arbing: price_sold.max - price_bought.min > 0.02, operating_profit>0    
    if (df_adr_arb[df_adr_arb['type_1']=='steth_sold']['steth_price'].max()-df_adr_arb[df_adr_arb['type_1']=='steth_bought']['steth_price'].min()) > 0.02 and operating_profit>0:
        list_resume_arbing_leaders.append(address)                                                                                                       

        
# sorting the lists by values         
dic_sort_by_value_of_exchange={}

for address in list_resume_arbing:
    dic_sort_by_value_of_exchange.update({address:dic_resume_all.get(address).iloc[0]['values']})
dic_sort_by_value_of_exchange={k: v for k, v in sorted(dic_sort_by_value_of_exchange.items(), reverse=True, key=lambda item: item[1])}


dic_sort_by_value_of_exchange_leaders={}

for address in list_resume_arbing_leaders:
    dic_sort_by_value_of_exchange_leaders.update({address:dic_resume_all.get(address).iloc[0]['values']})
dic_sort_by_value_of_exchange_leaders={k: v for k, v in sorted(dic_sort_by_value_of_exchange_leaders.items(), reverse=True, key=lambda item: item[1])}
   

In [4]:
# the outcome for addresses-leaders that made arbing
for address in dic_sort_by_value_of_exchange_leaders.keys():
    print('Address:', address)
    display(dic_resume_all.get(address))
    display(dic_trans.get(address))
    print()
    print()
    print()

Address: 0x40d227723d66eb5d5cb1dff427c51a49d0b9957e


,characteristics,values
0,"Value of exchange, stETH",1590.818500
1,"Margin of exchange, ETH",0.005491
2,"Gross profit, ETH:",8.734513
3,"APR, %:",7.645423
4,"Fees paid, ETH:",-0.324218
5,"Operating profit, ETH:",8.410295
6,"Net APR, %:",7.361631


,DateTime,Value_steth,Value_eth,TxnFee(ETH),steth_price,type_1
Txhash,,,,,,
0x6897a7e3a7d4c0a7eba14106a75b2f19c55e6a8f5943691c641b11faf0482950,2021-01-24 20:29:32,104.371567,-100.000000,-0.012968,0.958115,steth_bought
0x0549b6e99a8e9f2f5be29c8676275545493bd9f9f07d9ae54523b70e033492ed,2021-01-31 05:37:06,101.014453,-100.000000,-0.013651,0.989957,steth_bought
0x71ce5216adb98fb8b9ebdd20b7375b9d964b6e76cdbb9d2063126d2c907c7b84,2021-02-01 05:36:54,-100.152000,100.001346,-0.013553,0.998496,steth_sold
0x7b424e71cddc5de2dd3668a1516dccc79f7e041c920ca9308c59b763b8590077,2021-02-07 00:40:11,250.478704,-250.000000,-0.016182,0.998089,steth_bought
0xe1a6012fd4f03f54a024f85022b6d371e6f52dbec44eda8f4b79dd7ab338c4b0,2021-02-07 02:49:00,-250.000000,249.907013,-0.024642,0.999628,steth_sold
0x7f8be0dc5fd7e37afcb61d997b82b7c55ed7dae2b1f61df957f0af5b88b8c581,2021-02-09 22:58:05,200.448440,-200.000000,-0.026971,0.997763,steth_bought
0x9587e435bd7ef91f8af5304b0af356468fb1bc454b3acba172e9c39dc44511a1,2021-02-10 00:06:16,-200.126500,200.000055,-0.030805,0.999368,steth_sold
0x9c53c08512ec2c7d48fd7f9819f6a3d07e488d1c4bbd736b2f332ba909742613,2021-02-10 03:35:59,250.991916,-250.000000,-0.026971,0.996048,steth_bought
0x91bacd9d135a8c84bbdf9574538c6804b94a2e0b7c6a0c0d1beff4441e90310f,2021-02-13 00:22:43,-250.060000,250.004901,-0.034434,0.999780,steth_sold





Address: 0x0eb4add4ba497357546da7f5d12d39587ca24606


,characteristics,values
0,"Value of exchange, stETH",1029.729159
1,"Margin of exchange, ETH",0.027573
2,"Gross profit, ETH:",28.392890
3,"APR, %:",28.076531
4,"Fees paid, ETH:",-0.038585
5,"Operating profit, ETH:",28.354306
6,"Net APR, %:",28.038376


...

,characteristics,values
0,"Value of exchange, stETH",6.103101
1,"Margin of exchange, ETH",0.012453
2,"Gross profit, ETH:",0.076000
3,"APR, %:",12.174912
4,"Fees paid, ETH:",-0.032419
5,"Operating profit, ETH:",0.043580
6,"Net APR, %:",6.981435


,DateTime,Value_steth,Value_eth,TxnFee(ETH),steth_price,type_1
Txhash,,,,,,
0xd7991ffb709a596d0c38c79f56c70c60931a53dcef9af88f1739e8cdd1118d6a,2021-03-02 23:23:59,2.201905,-2.150000,-0.009785,0.976427,steth_bought
0xa0ab4a70d09a0d8a0688fcb4a0f71911d1d5b30926327dead60baf9569c1443d,2021-03-18 23:43:25,3.901196,-3.800000,-0.012528,0.974060,steth_bought
0x3b384ef18f474b361cd2f47b0195902f91640998cd9fa47e991c41cbbd1a1a3b,2021-03-31 07:48:25,-10.896144,10.692378,-0.019205,0.981299,steth_sold
0x5cf09589f87eaa3988e89f1516f324919bd605936d8cc3b3146be7c7d371d2cd,2021-04-09 07:22:54,-7.554659,7.525335,-0.011348,0.996118,steth_sold





Address: 0x11f262d4154f7860da5e4f595778087eb7434bce


,characteristics,values
0,"Value of exchange, stETH",2.565703
1,"Margin of exchange, ETH",0.026528
2,"Gross profit, ETH:",0.068063
3,"APR, %:",18.176711
4,"Fees paid, ETH:",-0.014914
5,"Operating profit, ETH:",0.053149
6,"Net APR, %:",14.193866


,DateTime,Value_steth,Value_eth,TxnFee(ETH),steth_price,type_1
Txhash,,,,,,
0xaf11c95415411270153761647ed54c44566eea38e7a31ae02d4901502368908c,2021-01-09 16:09:02,-4.958851,4.963412,-0.012267,1.000920,steth_sold
0xfec105642700866a6c99fab9e2c525203a3fcdb5c3c56ebc2ef49a7b6b42b122,2021-03-03 22:37:23,2.565703,-2.500000,-0.008567,0.974392,steth_bought


In [5]:
# the outcome for all addresses that made arbing
for address in dic_sort_by_value_of_exchange.keys():
    print('Address:', address)
    display(dic_resume_all.get(address))
    display(dic_trans.get(address))
    print()
    print()
    print()

Address: 0x40d227723d66eb5d5cb1dff427c51a49d0b9957e


,characteristics,values
0,"Value of exchange, stETH",1590.818500
1,"Margin of exchange, ETH",0.005491
2,"Gross profit, ETH:",8.734513
3,"APR, %:",7.645423
4,"Fees paid, ETH:",-0.324218
5,"Operating profit, ETH:",8.410295
6,"Net APR, %:",7.361631


,DateTime,Value_steth,Value_eth,TxnFee(ETH),steth_price,type_1
Txhash,,,,,,
0x6897a7e3a7d4c0a7eba14106a75b2f19c55e6a8f5943691c641b11faf0482950,2021-01-24 20:29:32,104.371567,-100.000000,-0.012968,0.958115,steth_bought
0x0549b6e99a8e9f2f5be29c8676275545493bd9f9f07d9ae54523b70e033492ed,2021-01-31 05:37:06,101.014453,-100.000000,-0.013651,0.989957,steth_bought
0x71ce5216adb98fb8b9ebdd20b7375b9d964b6e76cdbb9d2063126d2c907c7b84,2021-02-01 05:36:54,-100.152000,100.001346,-0.013553,0.998496,steth_sold
0x7b424e71cddc5de2dd3668a1516dccc79f7e041c920ca9308c59b763b8590077,2021-02-07 00:40:11,250.478704,-250.000000,-0.016182,0.998089,steth_bought
0xe1a6012fd4f03f54a024f85022b6d371e6f52dbec44eda8f4b79dd7ab338c4b0,2021-02-07 02:49:00,-250.000000,249.907013,-0.024642,0.999628,steth_sold
0x7f8be0dc5fd7e37afcb61d997b82b7c55ed7dae2b1f61df957f0af5b88b8c581,2021-02-09 22:58:05,200.448440,-200.000000,-0.026971,0.997763,steth_bought
0x9587e435bd7ef91f8af5304b0af356468fb1bc454b3acba172e9c39dc44511a1,2021-02-10 00:06:16,-200.126500,200.000055,-0.030805,0.999368,steth_sold
0x9c53c08512ec2c7d48fd7f9819f6a3d07e488d1c4bbd736b2f332ba909742613,2021-02-10 03:35:59,250.991916,-250.000000,-0.026971,0.996048,steth_bought
0x91bacd9d135a8c84bbdf9574538c6804b94a2e0b7c6a0c0d1beff4441e90310f,2021-02-13 00:22:43,-250.060000,250.004901,-0.034434,0.999780,steth_sold





Address: 0xcbba4fb07d4a86267741cf1a2c363284eed00f0f


,characteristics,values
0,"Value of exchange, stETH",1427.420346
1,"Margin of exchange, ETH",0.006760
2,"Gross profit, ETH:",9.649221
3,"APR, %:",12.668714
4,"Fees paid, ETH:",-0.125359
5,"Operating profit, ETH:",9.523863
6,"Net APR, %:",12.504128


...

,characteristics,values
0,"Value of exchange, stETH",0.013367
1,"Margin of exchange, ETH",0.010287
2,"Gross profit, ETH:",0.000138
3,"APR, %:",279.228902
4,"Fees paid, ETH:",-0.047808
5,"Operating profit, ETH:",-0.047670
6,"Net APR, %:",-96800.800636


,DateTime,Value_steth,Value_eth,TxnFee(ETH),steth_price,type_1
Txhash,,,,,,
0x6c1c5980d22fe04ef6a455a5e93bc078fb2b0b8f0cc7505f8c070a7a03e951db,2021-03-07 08:32:33,0.013367,-0.012811,-0.012108,0.958389,steth_bought
0xaba81e53c70990e743967bb81e55c6898a8360f1ca151f6016a9c0ced31ff2b8,2021-03-08 16:46:57,-0.000103,0.000100,-0.019410,0.968676,steth_sold
0x24c71dbcdb0303282cc18265117ba638e04d4f6258d98c143227221569deecd3,2021-03-08 16:48:53,-0.013270,0.012854,-0.016304,0.968676,steth_sold





Address: 0x7e58a5a589986aa3de2f29ee443b6dd1096c1901


,characteristics,values
0,"Value of exchange, stETH",1.000000e-04
1,"Margin of exchange, ETH",3.412879e-03
2,"Gross profit, ETH:",3.412879e-07
3,"APR, %:",6.896127e+01
4,"Fees paid, ETH:",-7.385952e-03
5,"Operating profit, ETH:",-7.385611e-03
6,"Net APR, %:",-1.492350e+06


,DateTime,Value_steth,Value_eth,TxnFee(ETH),steth_price,type_1
Txhash,,,,,,
0x13cf3f72b79aa6e1b886eb7de44deb374cd65f47c5ff681fc34cbca9e9b10a4e,2021-01-22 20:26:07,0.502449,-0.5000,-0.008491,0.995126,steth_bought
0xba0a131732f6ce5ee89142ca0a558953935a2fe100dabdc506c4317f0293d130,2021-01-22 22:20:52,0.602852,-0.6000,-0.007760,0.995270,steth_bought
0x02ec6bf121aaf7b6664468b47d1bda4f41aa00dddb58185f7b33155d2999f101,2021-01-24 15:47:18,-0.000100,0.0001,-0.007384,0.998617,steth_sold


In [6]:
# briefly for Notion
for address in dic_sort_by_value_of_exchange_leaders.keys():
    print('Address:', address)
    display(dic_resume_all.get(address))
    print()
    print()
    print()

Address: 0x40d227723d66eb5d5cb1dff427c51a49d0b9957e


,characteristics,values
0,"Value of exchange, stETH",1590.818500
1,"Margin of exchange, ETH",0.005491
2,"Gross profit, ETH:",8.734513
3,"APR, %:",7.645423
4,"Fees paid, ETH:",-0.324218
5,"Operating profit, ETH:",8.410295
6,"Net APR, %:",7.361631





Address: 0x0eb4add4ba497357546da7f5d12d39587ca24606


,characteristics,values
0,"Value of exchange, stETH",1029.729159
1,"Margin of exchange, ETH",0.027573
2,"Gross profit, ETH:",28.392890
3,"APR, %:",28.076531
4,"Fees paid, ETH:",-0.038585
5,"Operating profit, ETH:",28.354306
6,"Net APR, %:",28.038376





Address: 0x49cae5c4e03196bef2f9984469bd096878043719


...

,characteristics,values
0,"Value of exchange, stETH",15.472488
1,"Margin of exchange, ETH",0.021736
2,"Gross profit, ETH:",0.336307
3,"APR, %:",11.336835
4,"Fees paid, ETH:",-0.051061
5,"Operating profit, ETH:",0.285246
6,"Net APR, %:",9.615579





Address: 0xa91508dc7a7753ac2420adc2fbd17ce8364810e7


,characteristics,values
0,"Value of exchange, stETH",6.103101
1,"Margin of exchange, ETH",0.012453
2,"Gross profit, ETH:",0.076000
3,"APR, %:",12.174912
4,"Fees paid, ETH:",-0.032419
5,"Operating profit, ETH:",0.043580
6,"Net APR, %:",6.981435





Address: 0x11f262d4154f7860da5e4f595778087eb7434bce


,characteristics,values
0,"Value of exchange, stETH",2.565703
1,"Margin of exchange, ETH",0.026528
2,"Gross profit, ETH:",0.068063
3,"APR, %:",18.176711
4,"Fees paid, ETH:",-0.014914
5,"Operating profit, ETH:",0.053149
6,"Net APR, %:",14.193866
